In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import gym
from gym.spaces import Discrete, Tuple

from env_mp import MetaGames
from rmax_2 import RmaxAgent, Memory

In [2]:
K_epochs = 4  # update policy for K epochs
inner_gamma = 0.8  #inner game discount factor
meta_gamma = 0.8   #meta game discount factor
R_max = 0.98
max_meta_epi = 500
max_meta_steps = 50
max_inner_epi = 10
max_inner_steps = 5

epsilon=0.2
alpha = 0.4
radius=1

# creating environment
env = MetaGames("NL", "IPD")

#action_dim = env.d
nA = env.d
#state_dim = env.d * 2
nS = env.d * 2

#all agents' trajectory, entries will be like {[s1, a1, r1], [s2, a2, r2],...}
#torch size: {no. of inner episodes, no of inner steps, 3+no. of other agents*2 (2 since they take up 1 in state, 1 in rewards)}
traj=torch.empty(max_inner_epi, max_inner_steps, (3+(env.num_agents-1)*2)).to(device) 
our_rew = 0     #our agent's reward
oppo_rew = 0   #opponent's reward

memory = Memory()

In [5]:
state = env.reset()   #reset environment
dstate_index=[]
for j in torch.round(state, decimals = radius_dp):
    for i,x in enumerate(ref_arr):
        #find dstate index for each agent
        for num_agents in range(state.size(dim=1)):
            if x==j[num_agents-1]:
                dstate_index.append(i)
best_action = torch.argmax(innerq[:, dstate_index[0], :, 0], dim=1).unsqueeze(1)   

#run inner game according to that action, for K episodes & T timesteps, output used to be new_state, reward, done, _ 
newstate, reward, info, _ = env.step(best_action)  

traj[0,0] = torch.cat((state, best_action, reward, info), dim=1)
our_rew += reward.reshape(-1)
oppo_rew += info.reshape(-1)

#update inner r matrix
for i in range(env.b):
    #env.innerr[i, state[i], best_action[0,i], 0] += (inner_gamma**t) * reward[i] 
    #env.innerr[i, state[i], best_action[1,i], 1] += (inner_gamma**t) * info[i]
    env.innerr[i, state[i], best_action[0,i], 0] = reward[i] 
    env.innerr[i, state[i], best_action[1,i], 1] = info[i]

#update inner q matrix, another for loop since have to wait till inner r matrix gets updated    
for i in range(env.b):    
    env.innerq[i, state[i], best_action[0,i], 0] = env.innerr[i, state[i], best_action[0,i], 0] + inner_gamma * torch.max(env.innerq[i, newstate[i], :, 0]) 
    env.innerq[i, state[i], best_action[1,i], 1] = env.innerr[i, state[i], best_action[1,i], 1] + inner_gamma * torch.max(env.innerq[i, newstate[i], :, 1])

#find index of discretized state value, logic: match ref_arr to state values
state = newstate
# meta_s = torch.round(traj, decimals = radius_dp)                #meta-state = trajectory of all agents
# memory.states.append(meta_s)
# meta_a = torch.round(innerq[:, :, :, 0], decimals = radius_dp)  #meta-action = Q-table of inner game of our agent
# memory.actions.append(meta_a)
# meta_r = torch.round(our_rew, decimals = radius_dp)             #meta-reward = sum of inner rewards of our agent over K episodes & T timesteps
# memory.rewards.append(meta_r)

# best_meta_a = torch.max(meta_a)

NameError: name 'radius_dp' is not defined

In [ ]:
for episode in range(rmax.max_meta_epi): #for each meta-episode
    print("episode =", episode)
    #initialise meta-state and meta-action as zeros
    
    for step in range(rmax.max_meta_steps):    #for each meta time step
        print("timestep =", step)
        #reset environment
        env.reset()   
        #--------------------------------------START OF INNER GAME--------------------------------------            
        #for each inner episodes            
        for epi in range(max_inner_epi):                 
            for t in range(max_inner_steps):                     #for each inner timestep
                if t == 0:
                    #initialise action by random, of size [agents, batch_size]
                    best_action = env.init_action.detach().clone()  
                else:
                    #find action that has max Q value for current state for both agents
                    best_action = env.select_action().detach().clone()     
                
                #run inner game according to that action, for K episodes & T timesteps, output used to be new_state, reward, done, _ 
                reward, info = env.step(best_action.reshape(1))  
                
                #trajectory only comprises of [action, reward]
                traj[epi].append(best_action, [reward,info])      
                our_rew += reward.reshape(-1)
                oppo_rew += info.reshape(-1)

                #update inner r matrix
                env.innerr[0, best_action[0]] = reward
                env.innerr[1, best_action[1]] = info

                env.innerq[0, best_action[0]] = env.innerr[0, best_action[0]] + inner_gamma * torch.max(env.innerq[0, best_action[0]]) 
                env.innerq[1, best_action[1]] = env.innerr[1, best_action[1]] + inner_gamma * torch.max(env.innerq[1, best_action[1]])
        #---------------------------------------END OF INNER GAME---------------------------------                    
        #meta-state = inner game Q table for all agents
        meta_s = torch.round(traj, decimals = radius_dp).long().to(device)               
        memory.states.append(meta_s)
        
        #meta-action = inner game Q table for our agent ,YET TO DO: CREATE A MAPPING OF 1 AND 0 INSTEAD
        meta_a = touch.amax(env.innerq[:,:,:,0], dim=2)
        memory.actions.append(meta_a)
        
        #meta-reward = sum of inner rewards of our agent over K episodes & T timesteps
        meta_r = torch.round(our_rew, decimals = radius_dp)             
        memory.rewards.append(meta_r)
        
        #Rmax update step
        rmax.update(env, Memory, meta_s, meta_a_map, next_meta_s):
        


In [ ]:
        #find discretized state index that corresponds to the real state value
        dstate_index=[]
        for j in torch.round(state, decimals = radius_dp):
            for i,x in enumerate(ref_arr):
                if x==j:
                    dstate_index.append(i)